In [47]:
import pandas as pd
import requests
import collections
import json
import ast, sys
import urllib
import urllib2
import time

In [48]:
use_prod_db = True
use_proxy = True

if use_prod_db: 
    apikey_fc  = "4377190f-24a5-4ef7-a761-a877bf6218e3"
    base_url   = "https://www.feedcrunch.io/"
else:
    apikey_fc  = "a234ea9c-149d-4d28-8045-e2476153e571"
    base_url   = "http://172.21.159.208:5000/"

http_proxy  = "http://proxyweb.utc.fr:3128"
https_proxy = "http://proxyweb.utc.fr:3128"
ftp_proxy   = "http://proxyftp.utc.fr:3128"

proxyDict = { 
    "http"  : http_proxy, 
    "https" : https_proxy, 
    "ftp"   : ftp_proxy
}

if use_proxy:
    urllib_proxy = urllib2.ProxyHandler(proxyDict)
    opener = urllib2.build_opener(urllib_proxy)
    urllib2.install_opener(opener)

In [49]:
def remove_duplicate_rows(df, column_name_to_filter):
    grouped = df.groupby(column_name_to_filter)
    index = [gp_keys[0] for gp_keys in grouped.groups.values()]
    return df.reindex(index)

def convert_dict_unicode2string(data):
    if isinstance(data, basestring):
        return data.encode("utf8")
    elif isinstance(data, collections.Mapping):
        return dict(map(convert_dict_unicode2string, data.iteritems()))
    elif isinstance(data, collections.Iterable):
        return type(data)(map(convert_dict_unicode2string, data))
    else:
        return data
    
import mimetypes
import random
import string

_BOUNDARY_CHARS = string.digits + string.ascii_letters

def encode_multipart(fields, files, boundary=None):
    r"""Encode dict of form fields and dict of files as multipart/form-data.
    Return tuple of (body_string, headers_dict). Each value in files is a dict
    with required keys 'filename' and 'content', and optional 'mimetype' (if
    not specified, tries to guess mime type or uses 'application/octet-stream').

    >>> body, headers = encode_multipart({'FIELD': 'VALUE'},
    ...                                  {'FILE': {'filename': 'F.TXT', 'content': 'CONTENT'}},
    ...                                  boundary='BOUNDARY')
    >>> print('\n'.join(repr(l) for l in body.split('\r\n')))
    '--BOUNDARY'
    'Content-Disposition: form-data; name="FIELD"'
    ''
    'VALUE'
    '--BOUNDARY'
    'Content-Disposition: form-data; name="FILE"; filename="F.TXT"'
    'Content-Type: text/plain'
    ''
    'CONTENT'
    '--BOUNDARY--'
    ''
    >>> print(sorted(headers.items()))
    [('Content-Length', '193'), ('Content-Type', 'multipart/form-data; boundary=BOUNDARY')]
    >>> len(body)
    193
    """
    def escape_quote(s):
        return s.replace('"', '\\"')

    if boundary is None:
        boundary = ''.join(random.choice(_BOUNDARY_CHARS) for i in range(30))
    lines = []

    for name, value in fields.items():
        lines.extend((
            '--{0}'.format(boundary),
            'Content-Disposition: form-data; name="{0}"'.format(escape_quote(name)),
            '',
            str(value),
        ))

    for name, value in files.items():
        filename = value['filename']
        if 'mimetype' in value:
            mimetype = value['mimetype']
        else:
            mimetype = mimetypes.guess_type(filename)[0] or 'application/octet-stream'
        lines.extend((
            '--{0}'.format(boundary),
            'Content-Disposition: form-data; name="{0}"; filename="{1}"'.format(
                    escape_quote(name), escape_quote(filename)),
            'Content-Type: {0}'.format(mimetype),
            '',
            value['content'],
        ))

    lines.extend((
        '--{0}--'.format(boundary),
        '',
    ))
    body = '\r\n'.join(lines)

    headers = {
        'Content-Type': 'multipart/form-data; boundary={0}'.format(boundary),
        'Content-Length': str(len(body)),
    }

    return (body, headers)

In [50]:
def log_progress(sequence, every=None, size=None, name='Items'):
    from ipywidgets import IntProgress, HTML, VBox
    from IPython.display import display

    is_iterator = False
    if size is None:
        try:
            size = len(sequence)
        except TypeError:
            is_iterator = True
    if size is not None:
        if every is None:
            if size <= 200:
                every = 1
            else:
                every = int(size / 200)     # every 0.5%
    else:
        assert every is not None, 'sequence is iterator, set every'

    if is_iterator:
        progress = IntProgress(min=0, max=1, value=1)
        progress.bar_style = 'info'
    else:
        progress = IntProgress(min=0, max=size, value=0)
    label = HTML()
    box = VBox(children=[label, progress])
    display(box)

    index = 0
    try:
        for index, record in enumerate(sequence, 1):
            if index == 1 or index % every == 0:
                if is_iterator:
                    label.value = '{name}: {index} / ?'.format(
                        name=name,
                        index=index
                    )
                else:
                    progress.value = index
                    label.value = u'{name}: {index} / {size}'.format(
                        name=name,
                        index=index,
                        size=size
                    )
            yield record
    except:
        progress.bar_style = 'danger'
        raise
    else:
        progress.bar_style = 'success'
        progress.value = index
        label.value = "{name}: {index}".format(
            name=name,
            index=str(index or '?')
        )

## Formatting output from "Session Buddy"

In [51]:
with open('data.json') as f:
    string = ""
    for line in f:
        string += line
        
string = string.decode('ascii', 'ignore')
        
data = json.loads(unicode(string))
df = pd.DataFrame(data["tabs"])

df

,active,audible,autoDiscardable,discarded,favIconUrl,height,highlighted,id,incognito,index,mutedInfo,pinned,selected,status,title,url,width,windowId
0,False,False,True,False,http://engineering.flipboard.com/assets/favico...,920,False,1225,False,0,{u'muted': False},False,False,complete,Clustering Similar Stories Using LDA Flipboar...,http://engineering.flipboard.com/2017/02/story...,1680,1
1,False,False,True,False,https://assets-cdn.github.com/favicon.ico,871,False,1229,False,1,{u'muted': False},False,False,complete,bamos/densenet.pytorch: A PyTorch implementati...,https://github.com/bamos/densenet.pytorch?utm_...,1680,1
2,False,False,True,False,http://www.kdnuggets.com/wp-content/themes/kdn...,871,False,1237,False,2,{u'muted': False},False,False,complete,Deep Learning for Text Understanding from Scratch,http://www.kdnuggets.com/2015/03/deep-learning...,1680,1
3,False,False,True,False,http://media2.cdn.builtinchicago.org/sites/all...,871,False,1241,False,3,{u'muted': False},False,False,complete,The robot doctor will see you now: How AI coul...,http://www.builtinchicago.org/2016/08/05/reali...,1680,1
4,False,False,True,False,NaN,871,False,1245,False,4,{u'muted': False},False,False,complete,Unfolding RNNs,http://suriyadeepan.github.io/2017-01-07-unfol...,1680,1
5,False,False,True,False,NaN,871,False,1391,False,5,{u'muted': False},False,False,complete,Unfolding RNNs II,http://suriyadeepan.github.io/2017-02-13-unfol...,1680,1
6,False,False,True,False,http://www.andreykurenkov.com/images/favicon/f...,871,False,1405,False,6,{u'muted': False},False,False,complete,The Power of IPython Notebook + Pandas + and S...,http://www.andreykurenkov.com/writing/power-of...,1680,1
7,False,False,True,False,http://www.andreykurenkov.com/images/favicon/f...,871,False,1425,False,7,{u'muted': False},False,False,complete,A 'Brief' History of Neural Nets and Deep Lear...,http://www.andreykurenkov.com/writing/a-brief-...,1680,1
8,False,False,True,False,http://www.andreykurenkov.com/images/favicon/f...,871,False,1437,False,8,{u'muted': False},False,False,complete,A 'Brief' History of Neural Nets and Deep Lear...,http://www.andreykurenkov.com/writing/a-brief-...,1680,1
9,False,False,True,False,http://www.andreykurenkov.com/images/favicon/f...,871,False,1429,False,9,{u'muted': False},False,False,complete,A 'Brief' History of Neural Nets and Deep Lear...,http://www.andreykurenkov.com/writing/a-brief-...,1680,1


In [52]:
output_df = df[['title','url']]
output_df.insert( 2,'posting_user', "dataradar")
output_df.insert( 3,'tags',"")
output_df.insert( 4,'twitter',True)
output_df.insert( 5,'autoformat',False)
output_df.insert( 6,'activated',True)

output_df = output_df.rename(columns = {'url':'link'})

output_df = remove_duplicate_rows(output_df, 'link')

In [53]:
output_df.to_json("output.json", orient="records", force_ascii=False)

## Check Uniqueness in consolidated.json File

In [67]:
with open('consolidated.json') as f:
    string = ""
    for line in f:
        string += line
        
data = json.loads(string)
df = pd.DataFrame(data)

df = remove_duplicate_rows(df, 'link')

In [68]:
api_route = "api/1.0/authenticated/get/article/exists/"

api_url = base_url + api_route + apikey_fc + "/"

def check_link_exists(url, user):
    try:
        
        values = {
            'link' : url,
            'posting_user' : user
        }

        url_values  = urllib.urlencode(values)
        
        full_url = api_url + "?" + url_values
        
        response = urllib2.urlopen(full_url)
        
        return json.load(response)["exists"]
    
    except Exception as e:
        print "Error: " + str(e)
        return False

In [69]:
lines_2_delete = []

for index in log_progress(range(df.shape[0]), every=1):
    url = df.iloc[index]['link']
    user = df.iloc[index]['posting_user']
    
    if check_link_exists(url, user):
        lines_2_delete.append(index)
        print 'Duplicate Found: ' + url + " && user: " + user

Duplicate Found: http://www.pyimagesearch.com/2016/09/26/a-simple-neural-network-with-python-and-keras/ && user: dataradar
Duplicate Found: https://github.com/rushter/MLAlgorithms && user: dataradar


In [70]:
df.drop(df.index[lines_2_delete], inplace=True)

df = df.sort_values(by=['link', 'title'], ascending=[True, True])

In [71]:
df.to_json("consolidated.json", orient="records", force_ascii=False)

## Describe Data

In [72]:
with open('consolidated.json') as f:
    string = ""
    for line in f:
        string += line
        
data = json.loads(string)
df = pd.DataFrame(data)

print "There are " + str(df.shape[0]) + " articles in dataset"
df = remove_duplicate_rows(df, 'link')
print "There are " + str(df.shape[0]) + " articles in dataset without duplicates"

There are 381 articles in dataset
There are 381 articles in dataset without duplicates


## Sorting Data

In [18]:
with open('consolidated.json') as f:
    string = ""
    for line in f:
        string += line
        
data = json.loads(string)
df = pd.DataFrame(data)

df = remove_duplicate_rows(df, 'link') 
df = df.sort_values(by=['link', 'title'], ascending=[True, True])
#df = df.sort_values(by=['title', 'link'], ascending=[True, True])

df.to_json("consolidated.json", orient="records", force_ascii=False)
df

,activated,autoformat,link,posting_user,tags,title,twitter
0,True,False,http://101.datascience.community/2017/02/04/be...,engineering,,Best Practices for Machine Learning Engineering,False
1,True,False,http://172.21.132.49:8888/notebooks/sharedfold...,dataradar,,VAE-GAN-multi-gpu-celebA,True
2,True,False,http://ai.stanford.edu/~ronnyk/glossary.html,dataradar,,Glossary of Terms Journal of Machine Learning,True
3,True,False,http://andreas-zeller.blogspot.fr/2017/01/twel...,dataradar,,Andreas Zeller's Blog: Twelve LaTeX packages t...,True
4,True,False,http://andrewgelman.com/2017/02/07/can-fit-hid...,dataradar,,You can fit hidden Markov models in Stan - Sta...,True
5,True,False,http://beta-blog.archagon.net/2016/12/31/cheap...,dataradar,,Cheap and Painless eGPU Thrills on a 2013 MacB...,True
6,True,False,http://biorxiv.org/content/early/2017/02/07/10...,dataradar,,Approximate Bayesian Computation Algorithms fo...,True
7,True,False,http://blog.algorithmia.com/anaylze-tweets-top...,dataradar,,Introduction to Twitter Topic and Sentiment An...,True
8,True,False,http://blog.algorithmia.com/improve-saliency-d...,dataradar,,Improving the Accuracy of Saliency Detection -...,True
9,True,False,http://blog.datumbox.com/datumbox-machine-lear...,dataradar,,Datumbox Machine Learning Framework version 0....,True


## Hashtag Automation

In [ ]:
max_hashtags = 5

def hashtags_free_number(df, index):
    
    tags = df.iloc[index]['tags']
        
    str_list = tags.split(",")
    str_list = filter(None, str_list) # fastest
    
    return max_hashtags - len(str_list)

In [ ]:
def check_substr(input_str, keyword):
    
    tmp_str = str.lower(input_str)   
    tmp_keyword = str.lower(keyword)
    
    if keyword[-1:] == '-':
        
        tmp_str = str.replace(tmp_str, ' ', '')
        tmp_str = str.replace(tmp_str, '-', '')
        tmp_str = str.replace(tmp_str, '_', '')
        tmp_str = str.replace(tmp_str, '/', '')
        tmp_str = str.replace(tmp_str, '#', '')
        tmp_str = str.replace(tmp_str, '?', '')

        
        tmp_keyword = str.replace(tmp_keyword, ' ', '')
        tmp_keyword = str.replace(tmp_keyword, '-', '')
        tmp_keyword = str.replace(tmp_keyword, '_', '')
        tmp_keyword = str.replace(tmp_keyword, '/', '')
        tmp_keyword = str.replace(tmp_keyword, '#', '')
        tmp_keyword = str.replace(tmp_keyword, '?', '')
    
    
    if tmp_keyword in tmp_str:
        return True
    else:
        return False

In [ ]:
def check_pattern(df, index, pattern):
    
    if check_substr(df.iloc[index]['title'].encode("utf-8"), pattern):
        return True
    else:
        return False

In [ ]:
def set_keyword(df, index, keyword_list, pattern = ""):
     
    if not check_pattern(df, index, pattern):
        return False
        
    tags = df.iloc[index]['tags']
        
    str_list = tags.split(",")
    str_list = filter(None, str_list) # fastest
    
    str_list = [s.encode('ascii') for s in str_list] # unicode to string
    
    old_tmp = str_list[:]
    
    for keyword in keyword_list:
        if len(str_list) >= max_hashtags:
            break
        if keyword not in str_list:
            str_list.append(keyword)
    
    df.set_value(index, "tags", ",".join(str_list))
    return max_hashtags - len(str_list)

In [ ]:
def write_stats(df, posts_full, full_at_start):
    data = []
    total_posts = df.shape[0]
    print "Posts Full = " + str(posts_full)+"/"+str(df.shape[0])
    print "Posts Full at Start = " + str(full_at_start)+"/"+str(df.shape[0])

    for i in range (100):
        data.append(0)

    for index in range(df.shape[0]):
        tags = df.iloc[index]['tags']

        str_list = tags.split(",")
        str_list = filter(None, str_list) # fastest

        data[len(str_list)] += 1

    for i in range (100):
        if data[i] != 0:
            print "Number of rows with " + str(i) + " Hashtags : " + str(data[i]) + "/" + str(total_posts)

In [ ]:
##################### LOADING JSON ##################################
with open('consolidated.json') as f:
    string = ""
    for line in f:
        string += line
        
data = json.loads(string)
df = pd.DataFrame(data)

df = remove_duplicate_rows(df, 'url') 
df = df.sort_values(by=['url', 'title'], ascending=[True, True])

###################### PARSING JSON #################################
posts_full = 0
full_at_start = 0

for index in range(df.shape[0]):
    max_iteration = hashtags_free_number(df, index)
    
    if max_iteration > 0:
        for pattern, keywords_list in keywords.iteritems():  
            tmp_rslt = set_keyword(df, index, keywords_list, pattern)
            
            if tmp_rslt is not False:
                max_iteration = tmp_rslt
            
                if max_iteration <= 0:
                    break
        
        #"""
        if max_iteration > 0 and df.iloc[index]['user'] == "dataradar":
            
            Last_Keywords = ['Data', 'DataScience','AI']            
 
            tmp_rslt = set_keyword(df, index, Last_Keywords, "")

            if tmp_rslt is not False:
                max_iteration = tmp_rslt
        #"""
                        
        if max_iteration <= 0:
            posts_full += 1
        elif max_iteration == 5:
            print df.iloc[index]['title'] 
            print df.iloc[index]['tags']
            print "###########"
        
    else:
        full_at_start += 1
        posts_full += 1

write_stats(df, posts_full, full_at_start)

In [ ]:
df.to_json("consolidated.json", orient="records", force_ascii=False)
df

In [ ]:
keywords = {
    # DataRadar Oriented Tags 
    'Accuracy': ['Accuracy'],
    'Adversa': ['GAN', 'DeepLearning', 'MachineLearning', 'NeuralNet'],
    'Analysis': ['Analysis'],
    'Analytic': ['Analytic'],
    'Arxiv': ['Arxiv', 'Research'],
    'AutoEncoder': ['AE', 'AutoEncoder', 'DeepLearning', 'MachineLearning', 'NeuralNet'],
    'Bayes': ['Bayes', 'MachineLearning', 'Statistic'],
    'BigData-': ['BigData'],
    'Boosting': ['Boosting', 'MachineLearning'],
    'CNN ': ['CNN', 'ConvNet', 'DeepLearning', 'MachineLearning', 'NeuralNet'],
    'CUDA ': ['CUDA', 'GPU', 'NVidia'],
    'Caffe ': ['Caffe', 'DeepLearning', 'MachineLearning', 'NeuralNet', 'Library'],
    'Cassandra': ['Cassandra', 'Apache', 'BigData'],
    'Classifi': ['Classification', 'MachineLearning'],
    'Clustering': ['Clustering', 'MachineLearning'],
    'Cohort': ['Analysis', 'Cohort', 'Statistic'],
    'ComputerVision-': ['ComputerVision', 'MachineLearning'],
    'ConvNet': ['CNN', 'ConvNet', 'DeepLearning', 'MachineLearning', 'NeuralNet'],
    'Convol': ['CNN', 'ConvNet', 'DeepLearning', 'MachineLearning', 'NeuralNet'],
    'CrossValidation-': ['CrossValidation', 'Statistic', 'MachineLearning'],
    'CuDNN ': ['CuDNN', 'CUDA', 'GPU', 'NVidia'],
    'DNN ': ['DNN', 'DeepLearning', 'MachineLearning', 'NeuralNet'],
    'DSSTNE ': ['Amazon', 'DSSTNE', 'DeepLearning', 'MachineLearning', 'NeuralNet'],
    'Data': ['Data', 'DataScience'],
    'DataAnalysis-': ['DataAnalysis', 'MachineLearning', 'Statistic'],
    'DataAnalyst-': ['DataScientist', 'DataAnalyst', 'Job'],
    'DataLake-': ['DataLake', 'BigData'],
    'DataScience-': ['DataScience', 'Data'],
    'DataScientist-': ['DataScientist', 'DataAnalyst', 'Job'],
    'DataStructure-': ['DataStructure'],
    'DataViz-': ['DataViz'],
    'Dataset': ['Dataset'],
    'DeepDream-': ['DeepDream', 'Google', 'DeepLearning', 'MachineLearning', 'NeuralNet'],
    'DeepLearning-': ['DeepLearning', 'MachineLearning', 'NeuralNet', 'Library'],
    'Diagram': ['diagram', 'DataViz'],
    'Distribution': ['Statistic', 'Probability', 'Distribution'],
    'ElasticSearch-': ['ElasticSearch', 'BigData'],
    'EnsembleLearning-': ['EnsembleLearning', 'MachineLearning'],
    'EnsembleModel-': ['EnsembleLearning', 'MachineLearning'],
    'FPGA ': ['FPGA'],
    'FaceDetection-': ['FaceDetection', 'ComputerVision'],
    'FaceRecognition-': ['FaceRecognition', 'ComputerVision'],
    'FeatureExtract-': ['FeatureExtraction', 'MachineLearning'],
    'FeatureSelect-': ['FeatureSelection', 'MachineLearning'],
    'Flink ': ['Flink', 'Apache', 'BigData'],
    'Forecasting': ["Forecasting"],
    'GAN ': ['GAN', 'DeepLearning', 'MachineLearning', 'NeuralNet'],
    'Generali': ['Generalisation', 'MachineLearning'],
    'GenerativeModel-': ['GenerativeModel', 'MachineLearning'],
    'GettingStarted-': ['GettingStarted'],
    'Gradient': ['Gradient', 'MachineLearning'],
    'GradientBoosting-': ['GradientBoosting', 'MachineLearning'],
    'GradientDescent-': ['GradientDescent', 'MachineLearning'],
    'Guide': ['Guide'],
    'Hadoop': ['Hadoop', 'Apache', 'BigData'],
    'HowTo-': ['HowTo', 'GettingStarted'],
    'HyperParam-': ['HyperParameter', 'MachineLearning'],
    'ImageAnaly-': ['ImageAnalysis', 'ComputerVision', 'MachineLearning'],
    'ImageCaptio-': ['ImageCaptioning', 'ComputerVision', 'MachineLearning'],
    'ImageClassifi-': ['ImageClassification', 'ComputerVision', 'MachineLearning'],
    'ImageProcess-': ['ImageProcessing', 'ComputerVision', 'MachineLearning'],
    'ImageRecogni-': ['ImageRecognition', 'ComputerVision', 'MachineLearning'],
    'ImageSegment-': ['ImageSegmentation', 'ComputerVision', 'MachineLearning'],
    'ImageUnderst-': ['ImageUnderstanding', 'ComputerVision', 'MachineLearning'],
    'Kafka ': ['Kafka', 'Apache', 'Distributed', "Streaming"],
    'KMeans ': ['KMeans', 'MachineLearning', 'Clustering'],
    'KNN ': ['KNN', 'MachineLearning', 'Classification'],
    'Kaggle': ['Competition', 'Kaggle', 'MachineLearning'],
    'Keras ': ['DeepLearning', 'Keras', 'MachineLearning', 'NeuralNet', 'Library'],
    'LDA ': ['LDA', 'MachineLearning', 'FeatureSelection'],
    'LSTM ': ['DeepLearning', 'LSTM', 'MachineLearning', 'RNN', 'NeuralNet'],
    'Lasagne': ['DeepLearning', 'Lasagne', 'MachineLearning', 'NeuralNet', 'Library'],
    'MNIST ': ['MNIST'],
    'MXNet ': ['DeepLearning', 'MXNet', 'MachineLearning', 'NeuralNet', 'Library'],
    'MachineLearning-': ['MachineLearning'],
    'MariaDB': ['Database', 'MariaDB'],
    'Markov ': ['MarkovChain', 'MachineLearning', 'NeuralNet'],
    'Mathemati': ['Mathematic', 'Science'],
    'MissingVal-': ['MissingValue'],
    'Model': ['Model'],
    'MongoDB': ['Database', 'MongoDB'],
    'MonteCarlo-': ['MonteCarlo', 'Stochastic'],
    'Multilayer-': [' MultiLayer', 'DeepLearning', 'NeuralNet'],
    'MultiClass-': ['MultiClass', 'MachineLearning'],
    'MultiLabel-': ['MultiClass', 'MachineLearning'],
    'MySQL': ['Database', 'MySQL'],
    'NLP ': ['NLP', 'DeepLearning', 'MachineLearning'],
    'NVidia': ['NVidia'],
    'Neo4J': ['Neo4J', 'Database', 'GraphDB'],
    'Neural': ['NeuralNet', 'DeepLearning', 'MachineLearning'],
    'NeuralNet-': ['NeuralNet', 'DeepLearning', 'MachineLearning'],
    'NoSQL': ['NoSQL', 'Database', 'BigData'],
    'NonLinear-': ['NonLinear', 'MachineLearning'],
    'Numpy': ['Numpy', 'Python', 'Library'],
    'ObjectDetec-': ['ObjectDetection', 'ComputerVision', 'DeepLearning', 'MachineLearning'],
    'ObjectReco-': ['ObjectRecognition', 'ComputerVision', 'DeepLearning', 'MachineLearning'],
    'ObjectSegmen-': ['ObjectSegmentation', 'ComputerVision', 'DeepLearning', 'MachineLearning'],
    'Pandas': ['Pandas', 'Python', 'Library', 'DataManipulation'],
    'Pipeline': ['DataPipeline', 'MachineLearning'],
    'PostgreSQL': ['Database', 'PostgreSQL'],
    'Proba': ['Probability', 'Statistic'],
    'Propaga': ['BackPropagation', 'DeepLearning', 'NeuralNet'],
    'Python': ['Python'],
    'RSS ': ['RSS', 'Feed'],
    'Recurrent': ['DeepLearning', 'MachineLearning', 'RNN', 'NeuralNet'],
    'SVM ': ['MachineLearning', 'SVM', 'Classification'],
    'Scikit': ['Python', 'Scikit', 'MachineLearning', 'Library'],
    'Scipy': ['Python', 'Scipy', 'MachineLearning', 'Library'],
    'Segmentation': ["Segmentation"],
    'SelectFeatu-': ['FeatureSelection', 'MachineLearning'],
    'Sentiment': ['SentimentAnalysis', 'NLP', 'MachineLearning'],
    'Spark ': ['Apache', 'Spark', 'BigData'],
    'Streaming': ['Streaming', 'MachineLearning'],
    'TensorFlow': ['TensorFlow', 'DeepLearning', 'MachineLearning', 'NeuralNet', 'Library'],
    'Tensor': ['TensorFlow', 'DeepLearning', 'MachineLearning', 'NeuralNet', 'Library'],
    'TextMining-': ['TextMining', 'MachineLearning'],
    'TextProcess-': ['TextProcessing', 'MachineLearning'],
    'TimeSerie-': ["TimeSeries"],
    'Theano': ['DeepLearning', 'MachineLearning', 'Theano', 'Library'],
    'Tutorial': ['Tutorial', 'GettingStarted'],
    'Visuali': ['DataViz', 'MachineLearning'],
    'Vision': ['ComputerVision', 'MachineLearning'],
     
    # Engineering Oriented Tags 
    '2D': ["2D"],
    '3D': ['3D'],
    '3DModel-': ['3DModel', '3D'],
    'ABTest-': ['ABTest'],
    'API ': ['API'],
    'AWS ': ['AWS', 'Amazon', 'Cloud'],
    'AWSLambda-': ['AWSLambda', 'ServerLess', 'AWS', 'Amazon', 'Cloud'],
    'Academic': ['Academic', 'Research'],
    'Administration': ['Administration'],
    'AddOn-': ['AddOn'],
    'Adventure': ["Adventure"],
    'Agile': ['Agile'],
    'Algorithm': ['Algorithm', 'ComputerScience'],
    'Amazon': ['Amazon'],
    'AmazonWebService-': ['AWS', 'Amazon', 'Cloud', 'SaaS', 'PaaS'],
    'Android': ['Android', 'Mobile', 'Smartphone'],
    'Angular': ['Angular', 'JS', 'WebApp'],
    'Animation': ['Animation', 'CSS', 'WebDevelopment', 'WebApp'],
    'Approach': ['Approach'],
    'Async': ['Async'],
    'AutoScaling-': ['AutoScaling'],
    'Automation': ['Automation'],
    'Backdoor': ['Backdoor', 'Security', 'ITSec', 'InfoSec'],
    'Backup': ['Backup'],
    'Bash ': ['Bash', 'Script', 'Linux'],
    'Beautiful ':["Beautiful"],
    'BeautifulSoup-': ['BeautifulSoup', 'Python'],
    'Behavior': ['Behavior'],
    'Benchmark': ['Benchmark'],
    'Botnet ': ['Botnet', 'ITSecurity', 'InfoSec'],
    'Browser': ["Browser"],
    'Business': ['Business'],
    'Code ': ['Code', 'DevLife'],
    'C++ ': ['C++'],
    'CPU ': ['CPU'],
    'CSS ': ['CSS', 'WebApp'],
    'CSS3 ': ['CSS3'],
    'Cache ': ['Caching'],
    'Caching': ['Caching'],
    'Certificate': ['Certificate', 'Security', 'Encryption', 'ITSec', 'InfoSec'],
    'ChatBot-': ['ChatBot'],
    'Cheatsheet': ['Cheatsheet'],
    'CleanEnergy-': ["CleanEnergy", "GreenEnergy"],
    'CodeEditor-': ['CodeEditor','Code', 'DevLife'],
    'Collab': ['Collaborative'],
    'Compiler': ["Compiler"],
    'CommandLine-': ['CommandLine'],
    'Commit': ['Commit', 'Git', 'SVN'],
    'Compensation': ['Compensation', 'Gratification', 'Job', 'Salary'],
    'Computing': ['Computing', 'ComputerScience'],
    'Container': ['Container'],
    'Control': ['Control'],
    'CPython': ['CPython', 'Python', 'Performance', 'Optimization'],	 
    'Crypto': ['Cryptography', 'Security', 'Encryption'],
    'Customer': ["Customer"],
    'Dashboard': ["Dashboard"],
    'Decorator': ['Decorator', 'Code'],
    'DesignPattern-': ['DesignPattern', "Code", 'DevLife', 'BestPratice', 'SoftwareArchitecture'],
    'Design': ['Design'],
    'Developer': ['Developer', 'Job'],
    'Development': ['Development'],
    'DevelopmentCycle-': ["DevelopmentCycle"],
    'Django': ['Django', 'Python', 'WebApp', 'ORM'],
    'Docker': ['Docker', 'Container', 'Virtualisation', 'DataCenter', 'VM'],
    'Document': ['Document'],
    'Encrypt': ['Encryption', 'Security', 'ITSec', 'InfoSec'],
    'Energy': ["Energy"],
    'Excel': ['Excel', 'SpreadSheet', 'Microsoft', 'Office'],
    'Expert': ["Expert"],
    'Facebook': ['Facebook'],
    'Framework': ['Framework'],
    'Fundamental': ['Fundamental'],
    'FundRaising': ['FundRaising'],
    'Funding': ['Funding'],
    'GCE ': ['GCE', 'Google', 'Cloud'],
    'GCP ': ['GCP', 'Google', 'Cloud'],
    'GPU ': ['GPU'],
    'Galler': ['Gallery', 'Photo', 'Image'],
    'Git': ['Git', 'OpenSource'],
    'Github': ['Github', 'Git', 'OpenSource'],
    'Gitlab': ['Gitlab', 'Git', 'OpenSource'],
    'GoldenRule': ['GoldenRule'],
    'Google': ['Google'],
    'GoogleCloudPlatform-': ['GCP', 'Google', 'Cloud', 'SaaS', 'PaaS'],
    'GrowthHacking-': ['GrowthHacking'],
    'Hash ': ["Hashing", 'Hash', 'Cryptography', "ITSec", "InfoSec"],
    'HTML ': ['HTML', 'WebApp'],
    'HTML5 ': ["HTML5"],
    'HTTPS ': ['HTTPS', 'Encryption', 'Security'],
    'Handbook': ['Handbook'],
    'Haskell': ['Haskell'],
    'HumanCost-': ["HumanCost"],
    'IPython': ['IPython', 'Notebook', 'Python'],
    'Industry': ['Industry'],
    'Interactive': ['Interactive'],
    'InternetOfThi-': ['IoT'],
    'Interview': ['Interview'],
    'IoT ': ['IoT'],
    'Introduc': ['Introduction'],	 
    'JQuery': ['JQuery', 'JS', 'WebApp'],
    'JS ': ['JS', 'WebApp'],
    'JSON ': ['JSON'],
    'Java ': ['Java'],
    'Javascript': ['JS', 'WebApp'],
    'Jupyter': ['Jupyter', 'Notebook', 'Python'],
    'Kubernetes': ['Kubernetes', 'Docker','Orchestration'],
    'Language': ["Language"],
    'LaTeX': ['LaTeX'],
    'Log': ["Log"], 
    'Mac ': ['Mac', 'MacOS', 'Apple'],
    'MOOC': ['MOOC', 'Learning'],
    'MVP ': ['MVP', 'Startup'],
    'MagicLeap-': ['MagicLeap', 'VR', 'AR'],
    'MagicMirror-': ['MagicMirror'],
    'Markup': ['Markup'],
    'Memory': ['Memory', 'Performance'],
    'Messag': ['Message'],
    'Metasploit': ['Metasploit', 'Hack', 'Security', 'ITSec', 'InfoSec'],
    'Microservice-': ['MicroService', 'ServerLess', 'Cloud', 'SaaS'],
    'Microsoft': ['Microsoft'],
    'Migration': ['Migration', 'Database', 'ORM'],
    'Monitor': ['Monitoring', 'Service'],
    'Motion': ['Motion'],
    'NodeJS': ['NodeJS', 'JS', 'WebApp'],
    'Notebook': ['Notebook', 'Notebook', 'Python'],
    'Notif': ['Notification'],
    'OAuth': ['OAuth', 'Authentication'],
    'Online': ['Online'],
    'OpenCV': ['OpenCV', 'ComputerVision', 'C++', 'Python'],
    'OpenGL': ['OpenGL'],
    'OpenID': ['OpenID', 'Authentication'],
    'OpenSource-': ['OpenSource', 'Software'],
    'Opportunit': ["Opportunity"],
    'Orchestration': ["Orchestration"],
    'Personal': ['Personal'],
    'PHP ': ['PHP', 'WebApp'],
    'Pass': ['Password', 'Authentication'],
    'Performance': ['Performance'],
    'Pitch': ['Pitch', 'Startup', 'ElevatorPitch'],
    'Process': ['Process'],
    'Productiv': ['Productivity', 'Efficiency', 'Performance'],
    'Product ': ["Product"], # Don't remove the space
    'Programmer': ['Programmer', 'DevLife', 'ComputerScience'],
    'Programming': ["Programmer", 'DevLife', 'ComputerScience'],
    'Proof': ['Proof'],
    'Publish': ['Publishing'],
    'Push': ['Push'],
    'Radiobox': ["Radiobox"],
    'Reddit': ["Reddit"], 
    'REST ': ['REST'],
    'RSS ': ['RSS', 'Feed'],
    'Random': ['Random'],
    'Raspberry': ['RaspberryPi', 'CheapComputing', 'EmbeddedDevice'],
    'React': ['React', 'ReactJS', 'Javascript', 'WebApp', 'ReactNative'],
    'Recommendation': ['Recommendation', 'RecommendationEngine', 'RecommenderSystem'],
    'RegEx': ['Regex'],
    'RegularExpr-': ['Regex'],
    'RemoteAccess': ['RemoteAccess', 'RDP'],
    'Render': ['Render'],
    'Resilient': ['Resilient'],
    'SAML ': ['SAML', 'Authentication'],
    'SMS ': ['SMS', 'Message'],
    'Scala': ['Scala'],
    'Scientific': ['Scientific'],
    'Science': ['Science'],
    'Script': ['Script'],
    'Search': ['Search'],
    'SearchEngine-': ['SearchEngine'],
    'Securi': ['ITSec', 'Security', 'InfoSec'],
    'Server': ['Server'],
    'ServerLess-': ['ServerLess'],
    'Shell ': ['Shell'],
    'SideProject-': ['SideProject'],
    'Simple': ["Simple"],
    'Smart ': ['Smart'],
    'Software': ['Software'],
    'SSL ': ['SSL', 'Encryption', 'ITSec', 'InfoSec'],
    'Startup': ['Startup'],
    'Static': ['Static'],
    'Storage': ['Storage'],
    'SubString-': ['SubString'], 
    'Suggestion': ['Recommendation', 'RecommendationEngine', 'RecommenderSystem'],
    'System': ['System'],
    'Teach': ['Teach'],
    'TechDebt-': ['TechDebt'],
    'TechicalDebt-': ['TechDebt'],
    'Test': ['Test'],
    'Tethering': ['Tethering'],
    'Theory': ["Theory"],
    'Tool': ['Tool'],
    'Tracking': ['Tracking'],
    'UI ': ['UI', 'UX', 'Design'],
    'UX ': ['UI', 'UX', 'Design'],
    'VC ': ['VC', 'FundRaising', 'Investor'],
    'Visitor': ["Visitor"],
    'Web ': ['Web'], #Don't remove spaces
    'WebApp-': ['WebApp', 'WebDevelopment'],
    'WebDev-': ['WebDevelopment', 'WebApp'],
    'WebDesign': ["WebDesign", 'Design', 'UI', 'UX'],	
    'WebDesigner': ['WebDesigner', "WebDesign", 'Design', 'UI', 'UX'],
    'WebGL': ['WebGL'],
    'WebSocket-': ['WebSocket', 'Web', 'Async'],
    'Website': ['WebDevelopment', 'WebApp'],
    'Wireframe': ["Wireframe"],
}

## Uploading to FeedCrunch

In [ ]:
with open('consolidated.json') as f:
    string = ""
    for line in f:
        string += line
        
data = json.loads(string)
df = pd.DataFrame(data)

api_route = "api/1.0/authenticated/post/article/"

api_url = base_url + api_route + apikey_fc + "/"

restart_point = 0 # Starts at 0

for index in log_progress(range(restart_point, df.shape[0]), every=1):

    fields = convert_dict_unicode2string(df.iloc[index].to_dict())
    files = {}

    data, headers = encode_multipart(fields, files)
    request = urllib2.Request(api_url, data=data, headers=headers)

    f = urllib2.urlopen(request)

    tmp_rslt = f.read()
    
    try:
        tmp_rslt = tmp_rslt.replace('true', '"true"')
        tmp_rslt = tmp_rslt.replace('false', '"false"')

        data_rslt = ast.literal_eval(tmp_rslt)
        
        old_stdout = sys.stdout
        log_file = open("article_posting.log","a")
        sys.stdout = log_file
        
        if data_rslt["success"] != "true":
            print "Error  : " + str(index)
            print data_rslt["error"]
        else:
            print "Success: " + str(index)
        
        sys.stdout = old_stdout
        log_file.close()
            
        time.sleep(5 * 60) # Sleep for 5 Minutes
            
    except Exception, e:
        old_stdout = sys.stdout
        log_file = open("article_posting.log","a")
        sys.stdout = log_file
        print "#############"
        print "Error at index : " + str(index)
        print "An error occured in the process: " + str(e)
        sys.stdout = old_stdout
        log_file.close()
        break
        
    break